In [216]:
#Import necessary libraries

import csv
import requests
# !pip install geocoder
import geocoder
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim

In [217]:
html_doc=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(html_doc, 'html.parser')
tab_rows=soup.table.find_all('tr')


output_rows=[]
output_rows.append(['PostCode','Borough','Neighbourhood'])

for tr in tab_rows:
    td=tr.find_all('td')
    output_row=[]
    for i in td:
        output_row.append(i.text)
    output_rows.append(output_row)
    
with open('output.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(output_rows)

In [218]:
# Reading into a dataframe
df=pd.read_csv('output.csv')
df.head()

,PostCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


In [219]:
# Remove the /n
df['Neighbourhood']=df['Neighbourhood'].str.strip()
df.head()

,PostCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [220]:
# Remove entries with no values for Borough
df=df[df.Borough != 'Not assigned']
df.reset_index(drop=True,inplace=True)
df.head()

,PostCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


In [221]:
# Group with same PostCode

ind_drp=[]

for i in range(0,len(df['PostCode'])):
    for j in range(i+1,len(df['PostCode'])):
        if df['PostCode'][i]==df['PostCode'][j]:
            df['Neighbourhood'][i]=df['Neighbourhood'][i]+', '+df['Neighbourhood'][j]
            ind_drp.append(j)
            
df=df.drop(df.index[ind_drp])
df.reset_index(drop=True,inplace=True)

In [222]:
# Assign same values for missing Neighbourhood
for i in range(0,len(df['PostCode'])):
    if df['Neighbourhood'][i]=='Not assigned' :
            df['Neighbourhood'][i]=df['Borough'][i]

In [223]:
df.head()

,PostCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [224]:
df.shape

(103, 3)

In [226]:
dff=df

# Returns none, hence using the given csv file

# for i in range(0,len(df['PostCode'])):
#     lat_lng_coords = None
#     while lat_lng_coords is None :
#         g = geocoder.google('{}, Toronto, Ontario'.format(dff['PostCode'][i]))
#         lat_lng_coords = g.latlng

latlong=pd.read_csv('https://cocl.us/Geospatial_data')
latlong.rename(columns={'Postal Code':'PostCode'},inplace=True)
dff=dff.merge(latlong,on='PostCode')

dff.head(5)

,PostCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
